In [ ]:
import pandas as pd
import urllib.request
import urllib.parse
import xml.etree.ElementTree as ET
from tqdm import tqdm

spot_df = pd.read_csv('C:/ai_x/source/JikFam/data/RDA_SPOT_INFO.csv')
CropWeather_ServiceKey = open('C:/ai_x/source/JikFam/CropWeather_ServiceKey.txt', 'r').read().strip()
BASE_URL = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/V2/GnrlWeather/getWeatherMonDayList'

year_list = [str(y) for y in range(2018, 2026)]
month_list = [f"{m:02d}" for m in range(1, 13)]
last_month_2025 = 5

colname_dict = {
    'province': [],
    'obsr_Spot_Code': [],
    'date': [],
    'temp': [],
    'rain': [],
    'hum': []
}

error_list = []

for idx, row in tqdm(spot_df.iterrows(), total=spot_df.shape[0], desc="지점 반복"):
    stn_code = row['지점코드']
    stn_name = row['지점명']
    province = row['도명']
    start_date = pd.to_datetime(row['관측시작일'])

    for year in tqdm(year_list, leave=False, desc=f"{province} 연도"):
        for month in month_list:
            if year == '2025' and int(month) > last_month_2025:
                continue  # 2025년 5월까지만

            this_date = pd.to_datetime(f"{year}-{month}-01")
            if this_date < start_date:
                continue  # 관측시작일 이전 skip

            params = {
                'serviceKey': CropWeather_ServiceKey,
                'Page_No': '1',
                'Page_Size': '31',
                'search_Year': year,
                'search_Month': month,
                'obsr_Spot_Nm': stn_name,
                'obsr_Spot_Code': stn_code
            }

            url = BASE_URL + '?' + urllib.parse.urlencode(params)

            try:
                response = urllib.request.urlopen(url).read()
                response_string = ET.fromstring(response)

                body = response_string.find('body')
                if body is None:
                    raise ValueError("body가 없음 (데이터가 없습니다).")

                items = body.find('items')
                if items is None:
                    raise ValueError("items가 없음 (데이터가 없습니다).")

                item_list = items.findall('item')
                if not item_list:
                    raise ValueError("item 리스트가 없음 (데이터가 없습니다).")

                for item in item_list:
                    colname_dict['province'].append(province)
                    colname_dict['obsr_Spot_Code'].append(stn_code)
                    colname_dict['date'].append(item.findtext('date'))
                    colname_dict['temp'].append(item.findtext('temp'))
                    colname_dict['rain'].append(item.findtext('rain'))
                    colname_dict['hum'].append(item.findtext('hum'))

            except Exception as e:
                error_list.append({
                    'year': year,
                    'month': month,
                    'stn_code': stn_code,
                    'province': province,
                    'url': url,
                    'error': str(e)
                })

weather_df = pd.DataFrame(colname_dict)

for col in ['temp', 'rain', 'hum']:
    weather_df[col] = pd.to_numeric(weather_df[col], errors='coerce')

weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df.to_csv('농업기상데이터_2018_2025.csv', index=False, encoding='utf-8-sig')
print("✅ 기상 데이터 CSV 저장 완료!")

if error_list:
    error_df = pd.DataFrame(error_list)
    error_df.to_csv('기상데이터_수집에러.csv', index=False, encoding='utf-8-sig')
    print("에러 로그 저장 완료")
else:
    print("에러 없음")
